# Synthetic Academic Query Generation with OpenAlex

This notebook demonstrates how to generate diverse academic queries using the OpenAlex API. The system creates realistic queries that can be used for testing lead generation systems.

## Features

- **Diverse Query Generation**: Uses OpenAlex concept hierarchy and institution data
- **Configurable Parameters**: Target number, batch size, concept levels, etc.
- **Checkpointing**: Saves progress and can resume from interruptions
- **Batching**: Processes queries in batches for efficiency
- **Results with Names and Institutions**: Generates structured lead data

## Quick Start


In [1]:
import os
import sys
import asyncio
from pathlib import Path

# Add the src directory to the path
sys.path.append("../src")

from src.evals.generate_synthetic_questions import (
    GenerationConfig,
    SyntheticQueryGenerator,
)
from rich import print as rprint
import json
import pandas as pd

## Optional: Clean Up Old Checkpoints

If you're getting validation errors from old checkpoints, you can clean them up:


In [ ]:
# Optional: Clean up old checkpoints if you're getting validation errors
import shutil
from pathlib import Path

checkpoint_path = Path("checkpoints/demo_queries")
if checkpoint_path.exists():
    shutil.rmtree(checkpoint_path)
    print("Cleaned up old checkpoints")
else:
    print("No old checkpoints to clean")

## Configuration

First, let's set up the configuration for our query generation. You can adjust these parameters based on your needs.


In [ ]:
# Configure email for OpenAlex API (recommended)
# You can set this as an environment variable: OPENALEX_EMAIL
# os.environ['OPENALEX_EMAIL'] = 'your.email@example.com'

# Create configuration
config = GenerationConfig(
    target_queries=300,  # Start small for testing
    batch_size=100,
    max_results_per_query=5,
    checkpoint_dir="checkpoints/demo_queries",
    output_file="eval_leads_v1.json",
    parallel_batch_size=6,
)

print("Configuration:")
print(f"Target queries: {config.target_queries}")
print(f"Batch size: {config.batch_size}")
print(f"Max results per query: {config.max_results_per_query}")
print(f"Output file: {config.output_file}")

## Generate Synthetic Queries

Now let's run the query generation. This will:

1. Fetch concepts and institutions from OpenAlex
2. Generate diverse query variations
3. Execute queries against OpenAlex
4. Save results with checkpointing

**Note**: This may take several minutes depending on your target number of queries.


In [ ]:
# Create the generator
generator = SyntheticQueryGenerator(config)

# Run the generation
await generator.gather_data()

print(generator.topics_per_institution)

In [ ]:
# Create the generator
generator = SyntheticQueryGenerator(config)

# Run the generation
results = await generator.generate_queries()